# Customized CNN inference
This notebook will take you to inference a 18 class CNN model.

## Prepare the overlay
We will download the overlay onto the board.

In [1]:
from pynq_dpu import DpuOverlay
overlay = DpuOverlay("dpu.bit")

## Import commom libraries

In [2]:
import os
import cv2
import time
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Load pretrained model (.xmodel)
The `load_model()` method will automatically prepare the graph which is used by VART.

In [3]:
overlay.load_model("lane_following.xmodel")

## Utility functions
We will prepare a few useful preprocessing functions for later use.

In [4]:
def BGR2RGB(image):
    B, G, R = cv2.split(image)
    image = cv2.merge([R, G, B])
    return image

def normalize(image):
    return np.asarray(image/255.0, dtype=np.float32)

def calculate_softmax(data):
    result = np.exp(data)
    return result

def preprocess_fn(image):
    image = BGR2RGB(image)
    image = normalize(image)
    return image

## Prepare inference dataset

In [5]:
!unzip -o -q dataset.zip

In [6]:
labels = []
image_path = []
total_images = 0

image_folder = 'dataset/test_data'
dirs = os.listdir(image_folder)
dirs.sort()
for d in dirs:
    labels.append(d)
    tmp_path = []
    dirs1 = os.listdir(image_folder + '/' + d)
    dirs1.sort()
    for d2 in dirs1:
        tmp_path.append(image_folder + '/' + d + '/' + d2)
    image_path.append(tmp_path)
    total_images += len(tmp_path)

print("Total number of test images: {}".format(total_images))
print("Dimension of each picture: {}x{}".format(cv2.imread(image_path[0][0]).shape[0], 
                                                cv2.imread(image_path[0][0]).shape[1]))
print("Labels: {}\n{}".format(len(labels), labels))

Total number of test images: 852
Dimension of each picture: 480x640
Labels: 18
['L_1', 'L_2', 'L_3', 'L_4', 'L_5', 'L_6', 'L_7', 'L_8', 'R_1', 'R_2', 'R_3', 'R_4', 'R_5', 'R_6', 'R_7', 'R_8', 'S_L', 'S_R']


## Use VART
Now we should be able to use *VART* to do the inference.

In [7]:
dpu = overlay.runner

inputTensors = dpu.get_input_tensors()
outputTensors = dpu.get_output_tensors()

shapeIn = tuple(inputTensors[0].dims)
shapeOut = tuple(outputTensors[0].dims)

We can define a few buffers to store input and output data. They will be reused during multiple runs.

In [8]:
input_data = [np.empty(shapeIn, dtype=np.float32, order="C")]
output_data = [np.empty(shapeOut, dtype=np.float32, order="C")]

## Run DPU to make predictions
We make predictions over the entire test dataset and calculate accuracy and performance.

In [ ]:
print("Evaluating {} images ...".format(total_images))
correct = 0

start = time.time()
for label in range(len(labels)):
    for image_index in range(len(image_path[label])):
        
        test_data = cv2.imread(image_path[label][image_index])
        input_data[0] = preprocess_fn(test_data)
        
        job_id = dpu.execute_async(input_data, output_data)
        dpu.wait(job_id)
        
        probs = calculate_softmax(output_data[0])
        predict = np.argmax(probs)
        correct += (predict == label)

stop = time.time()
execution_time = stop - start
print("Overall accuracy: {:.2f}%".format((correct/total_images)*100))
print("  Execution time: {:.4f}s".format(execution_time))
print("     Performance: {:.4f}FPS".format(total_images/execution_time))

Evaluating 852 images ...


## Randomly select a picture to predict

In [ ]:
def runOnce(labels, image_path):
    
    label = random.randint(0, len(labels)-1)
    num_label_images = len(image_path[label])
    image_index = random.randint(0, num_label_images-1)
    
    start = time.time()
    
    test_data = cv2.imread(image_path[label][image_index])
    input_data[0] = preprocess_fn(test_data)
    
    job_id = dpu.execute_async(input_data, output_data)
    dpu.wait(job_id)
    
    probs = calculate_softmax(output_data[0])
    predict = np.argmax(probs)
    
    stop = time.time()
    execution_time = stop - start
    
    print("Image label: {}".format(labels[label]))
    print("Predicted label: {}".format(labels[predict]))
    print("Execution time: {:.4f}s".format(execution_time))
    
    plt.imshow(cv2.cvtColor(test_data, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
runOnce(labels, image_path)

## Clean up
We will need to remove references to *vart.Runner*. This will make sure we can run other notebooks without any issue.

In [ ]:
del overlay
del dpu